<a href="https://colab.research.google.com/github/chasubeen/TalesRunnner/blob/doeun/Image_Captioning_BLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image-Captioning-BLIP

## 코드 개요
BLIP 모델을 사용해 training & validation 데이터셋의 이미지로부터 캡션을 생성하고, 생성된 캡션을 한국어로 번역하여 지정된 CSV 파일에 저장 및 관리하는 코드.

## 작동 흐름
1. 환경 설정: 필요한 라이브러리와 파일 경로를 설정
2. 데이터 준비: 데이터셋 압축 해제 후 jpg 파일 필터링
3. 모델 및 프로세서 로드: BLIP 모델 및 프로세서 로드
4. 캡션 생성: BLIP 모델로 이미지 캡션 생성 및 GoogleTranslator로 한국어 번역 수행 후 결과물을 csv 파일로 저장
5. 작업 중단 시 대처: 캡션 생성 도중 작업 중단 시 대처 관련 코드
6. 최종 데이터 통계 확인: 데이터 전처리 완료 후 전체 데이터 개수 확인
    - Training: 40001 개
    - Validation: 5000 개

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers
!pip install deep-translator

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration # 이미지 캡셔닝
from deep_translator import GoogleTranslator # 번역기

import os
import csv
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# DecompressionBombError 방지: 최대 이미지 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

## 2. 데이터 준비

- **데이터/이미지 경로 설정**  
  - `zip_folder_path`: 데이터 zip 파일 위치  
  - `extraction_base_path`: 압축 해제 경로 (가상 메모리에 저장)  

- **데이터 압축 해제**  
  - `unzip`을 사용해 zip 파일 해제  

- **파일 정리**  
  - `.json`과 `.jpg` 파일이 함께 존재 → `.jpg` 파일만 유지하여 처리 속도 개선  

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 특정 데이터셋만 압축 해제

In [ ]:
# ZIP 파일이 있는 폴더 경로
zip_folder_path = '/content/drive/My Drive/7th-project/data/Training'
extraction_base_path = '/content/data'

# 압축 해제할 특정 ZIP 파일 리스트 (확장자 포함)
selected_zip_files = ['TL_01T_의사소통_03S_초등_고학년.zip', 'TL_02T_자연탐구_01S_유아.zip', 'TL_02T_자연탐구_02S_초등_저학년.zip', 'TL_02T_자연탐구_03S_초등_고학년.zip']

# ZIP 파일 순회
for file_name in selected_zip_files:
    zip_file_path = os.path.join(zip_folder_path, file_name)

    # ZIP 파일이 존재하는지 확인
    if os.path.exists(zip_file_path):
        # 파일명과 동일한 디렉토리 생성
        extract_dir_name = os.path.splitext(file_name)[0]  # 확장자 제거한 이름
        extract_dir_path = os.path.join(extraction_base_path, extract_dir_name)
        os.makedirs(extract_dir_path, exist_ok=True)  # 디렉토리 생성

        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir_path)

        print(f"압축 해제 완료: {file_name} -> {extract_dir_path}")
    else:
        print(f"ZIP 파일이 존재하지 않음: {file_name}")

print("✅ 모든 선택된 ZIP 파일 압축 해제 완료!")


압축 해제 완료: TL_01T_의사소통_03S_초등_고학년.zip -> /content/data/TL_01T_의사소통_03S_초등_고학년
압축 해제 완료: TL_02T_자연탐구_01S_유아.zip -> /content/data/TL_02T_자연탐구_01S_유아
압축 해제 완료: TL_02T_자연탐구_02S_초등_저학년.zip -> /content/data/TL_02T_자연탐구_02S_초등_저학년
압축 해제 완료: TL_02T_자연탐구_03S_초등_고학년.zip -> /content/data/TL_02T_자연탐구_03S_초등_고학년
✅ 모든 선택된 ZIP 파일 압축 해제 완료!


### Training 데이터셋 압축 해제

In [ ]:
### Training 데이터셋 압축 해제

# ZIP 파일들이 있는 폴더 경로 설정
zip_folder_path = '/content/drive/My Drive/7th-project/data/Training'
extraction_base_path = '/content/data'

# ZIP 파일명을 저장할 리스트
zip_file_names = []

# ZIP 파일 순회
for file_name in os.listdir(zip_folder_path):
    # 파일이 ZIP 파일인지 확인
    if file_name.endswith('.zip'):
        zip_file_names.append(file_name)  # 파일명 추가
        zip_file_path = os.path.join(zip_folder_path, file_name)

        # 파일명과 동일한 디렉토리 생성
        extract_dir_name = os.path.splitext(file_name)[0]  # 확장자 제거한 이름
        extract_dir_path = os.path.join(extraction_base_path, extract_dir_name)
        os.makedirs(extract_dir_path, exist_ok=True)  # 디렉토리 생성

        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir_path)
        print(f"압축 해제 완료: {file_name} -> {extract_dir_path}")

# ZIP 파일명 리스트 출력
print("처리된 ZIP 파일 리스트:", zip_file_names)

압축 해제 완료: TL_05T_신체운동_건강_03S_초등_고학년.zip -> /content/data/TL_05T_신체운동_건강_03S_초등_고학년
압축 해제 완료: TL_04T_예술경험_03S_초등_고학년.zip -> /content/data/TL_04T_예술경험_03S_초등_고학년
압축 해제 완료: TL_05T_신체운동_건강_01S_유아.zip -> /content/data/TL_05T_신체운동_건강_01S_유아
압축 해제 완료: TL_04T_예술경험_02S_초등_저학년.zip -> /content/data/TL_04T_예술경험_02S_초등_저학년
압축 해제 완료: TL_05T_신체운동_건강_02S_초등_저학년.zip -> /content/data/TL_05T_신체운동_건강_02S_초등_저학년
압축 해제 완료: TL_02T_자연탐구_03S_초등_고학년.zip -> /content/data/TL_02T_자연탐구_03S_초등_고학년
압축 해제 완료: TL_02T_자연탐구_02S_초등_저학년.zip -> /content/data/TL_02T_자연탐구_02S_초등_저학년
압축 해제 완료: TL_03T_사회관계_01S_유아.zip -> /content/data/TL_03T_사회관계_01S_유아
압축 해제 완료: TL_04T_예술경험_01S_유아.zip -> /content/data/TL_04T_예술경험_01S_유아
압축 해제 완료: TL_01T_의사소통_01S_유아.zip -> /content/data/TL_01T

In [ ]:
### Training 파일 처리

# 딕셔너리 초기화
directory_file_count = {}

for subdir, _, files in os.walk(extraction_base_path):
    total_files = len(files)  # 원래 전체 파일 개수
    jpg_files = 0  # .jpg 파일 개수 초기화

    for filename in files:
        file_path = os.path.join(subdir, filename)

        # .json 파일 삭제
        if filename.endswith('.json'):
            os.remove(file_path)  # 파일 삭제
        # .jpg 파일 개수 세기
        elif filename.endswith('.jpg'):
            jpg_files += 1

    # 딕셔너리에 저장
    directory_file_count[subdir] = jpg_files

    # 결과 출력
    print(f"디렉토리: {subdir}")
    print(f".jpg 파일 개수: {jpg_files} / 원래 전체 파일 개수: {total_files}")
    print("-" * 50)

디렉토리: /content/data
.jpg 파일 개수: 0 / 원래 전체 파일 개수: 0
--------------------------------------------------
디렉토리: /content/data/TL_01T_의사소통_01S_유아
.jpg 파일 개수: 2483 / 원래 전체 파일 개수: 4966
--------------------------------------------------
디렉토리: /content/data/TL_05T_신체운동_건강_02S_초등_저학년
.jpg 파일 개수: 1496 / 원래 전체 파일 개수: 2992
--------------------------------------------------
디렉토리: /content/data/TL_03T_사회관계_02S_초등_저학년
.jpg 파일 개수: 5371 / 원래 전체 파일 개수: 10742
--------------------------------------------------
디렉토리: /content/data/TL_04T_예술경험_02S_초등_저학년
.jpg 파일 개수: 1308 / 원래 전체 파일 개수: 2616
--------------------------------------------------
디렉토리: /content/data/TL_03T_사회관계_01S_유아
.jpg 파일 개수: 1125 / 원래 전체 파일 개수: 2250
--------------------------------------------------
디렉토리: /content/data/TL_05T_신체운동_건강_03S_초등_고학년
.jpg 파일 개수: 1595 / 원래 전체 파일 개수: 3190
--------------------------------------------------
디렉토리: /content/data/TL_04T_예술ᄀ

### Validation 데이터셋 압축 해제

In [ ]:
### Validation 데이터셋 압축 해제

# Validation 데이터셋 폴더 경로 설정
zip_folder_path = '/content/drive/My Drive/7th-project/data/Validation'
extraction_base_path = '/content/data'

# ZIP 파일명을 저장할 리스트
zip_file_names = []

# ZIP 파일 순회
for file_name in os.listdir(zip_folder_path):
    # 파일이 ZIP 파일인지 확인
    if file_name.endswith('.zip'):
        zip_file_names.append(file_name)  # 파일명 추가
        zip_file_path = os.path.join(zip_folder_path, file_name)

        # 파일명과 동일한 디렉토리 생성
        extract_dir_name = os.path.splitext(file_name)[0]  # 확장자 제거한 이름
        extract_dir_path = os.path.join(extraction_base_path, extract_dir_name)
        os.makedirs(extract_dir_path, exist_ok=True)  # 디렉토리 생성

        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir_path)
        print(f"압축 해제 완료: {file_name} -> {extract_dir_path}")

# ZIP 파일명 리스트 출력
print("처리된 ZIP 파일 리스트:", zip_file_names)

압축 해제 완료: VL_02T_자연탐구_03S_초등_고학년.zip -> /content/data/VL_02T_자연탐구_03S_초등_고학년
압축 해제 완료: VL_05T_신체운동_건강_03S_초등_고학년.zip -> /content/data/VL_05T_신체운동_건강_03S_초등_고학년
압축 해제 완료: VL_04T_예술경험_03S_초등_고학년.zip -> /content/data/VL_04T_예술경험_03S_초등_고학년
압축 해제 완료: VL_05T_신체운동_건강_02S_초등_저학년.zip -> /content/data/VL_05T_신체운동_건강_02S_초등_저학년
압축 해제 완료: VL_04T_예술경험_02S_초등_저학년.zip -> /content/data/VL_04T_예술경험_02S_초등_저학년
압축 해제 완료: VL_05T_신체운동_건강_01S_유아.zip -> /content/data/VL_05T_신체운동_건강_01S_유아
압축 해제 완료: VL_02T_자연탐구_02S_초등_저학년.zip -> /content/data/VL_02T_자연탐구_02S_초등_저학년
압축 해제 완료: VL_03T_사회관계_01S_유아.zip -> /content/data/VL_03T_사회관계_01S_유아
압축 해제 완료: VL_04T_예술경험_01S_유아.zip -> /content/data/VL_04T_예술경험_01S_유아
압축 해제 완료: VL_01T_의사소통_03S_초등_고학년.zip -> /content/d

In [ ]:
### Validation 파일 처리

# 딕셔너리 초기화
directory_file_count = {}

for subdir, _, files in os.walk(extraction_base_path):
    total_files = len(files)  # 원래 전체 파일 개수
    jpg_files = 0  # .jpg 파일 개수 초기화

    for filename in files:
        file_path = os.path.join(subdir, filename)

        # .json 파일 삭제
        if filename.endswith('.json'):
            os.remove(file_path)  # 파일 삭제
        # .jpg 파일 개수 세기
        elif filename.endswith('.jpg'):
            jpg_files += 1

    # 결과 출력
    print(f"디렉토리: {subdir}")
    print(f".jpg 파일 개수: {jpg_files} / 원래 전체 파일 개수: {total_files}")
    print("-" * 50)

디렉토리: /content/data
.jpg 파일 개수: 0 / 원래 전체 파일 개수: 0
--------------------------------------------------
디렉토리: /content/data/VL_03T_사회관계_01S_유아
.jpg 파일 개수: 152 / 원래 전체 파일 개수: 304
--------------------------------------------------
디렉토리: /content/data/VL_05T_신체운동_건강_02S_초등_저학년
.jpg 파일 개수: 171 / 원래 전체 파일 개수: 342
--------------------------------------------------
디렉토리: /content/data/VL_03T_사회관계_03S_초등_고학년
.jpg 파일 개수: 442 / 원래 전체 파일 개수: 884
--------------------------------------------------
디렉토리: /content/data/VL_05T_신체운동_건강_03S_초등_고학년
.jpg 파일 개수: 225 / 원래 전체 파일 개수: 450
--------------------------------------------------
디렉토리: /content/data/VL_02T_자연탐구_03S_초등_고학년
.jpg 파일 개수: 163 / 원래 전체 파일 개수: 326
--------------------------------------------------
디렉토리: /content/data/VL_01T_의사소통_01S_유아
.jpg 파일 개수: 358 / 원래 전체 파일 개수: 716
--------------------------------------------------
디렉토리: /content/data/VL_04T_예술경험_03S_초드

## 3. 모델 및 프로세서 로드

- 모델: BLIP

In [ ]:
# BLIP 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
### 캡션 생성 함수
def generate_korean_caption(image_path):

    # 이미지 로드
    image = Image.open(image_path).convert('RGB')

    # BLIP 입력 생성
    inputs = processor(image, return_tensors="pt")

    # 영어 캡션 생성
    outputs = model.generate(**inputs)
    caption_eng = processor.decode(outputs[0], skip_special_tokens=True)

    # 영어 캡션을 한국어로 번역
    caption_kor = GoogleTranslator(source='en', target='ko').translate(caption_eng)

    return image, caption_eng, caption_kor

In [ ]:
### 디버깅용: 이미지 시각화 함수
def display_image(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

## 캡션 생성

**[이미지 처리 및 CSV 업데이트 함수]**

- 입력된 이미지에서 캡션을 생성하고, 한국어로 번역
- 이미지 파일명(`id`), 이미지 경로(`img_path`), 한국어 캡션(`caption`)을 포함한 데이터 구성
- 기존 CSV 파일이 있으면 업데이트, 없으면 새로 생성하여 데이터 저장

In [ ]:
### CSV 파일 경로 설정: Training vs Validation

# Training
csv_file_path = '/content/drive/My Drive/7th-project/data/captions_train.csv'
# Validation
# csv_file_path = '/content/drive/My Drive/7th-project/data/captions_val.csv'

In [ ]:
### 이미지 처리 및 CSV 파일 생성/업데이트 함수

def process_images_and_update_csv(dir_path, file_name, csv_file_path):
    # 이미지 파일 전체 경로
    image_path = os.path.join(dir_path, file_name)

    # 캡션 생성
    image, caption_eng, caption_kor = generate_korean_caption(image_path)

    # 디버깅: 캡션 출력
    print(f"image_path: {image_path}, caption: {caption_kor}")

    # 파일명에서 확장자 제거
    file_id = os.path.splitext(file_name)[0]

    # 데이터 준비
    entry = {
        'file_name': file_id,
        'img_path': image_path,
        'caption': caption_kor
    }

    # CSV 파일 생성 또는 업데이트
    file_exists = os.path.exists(csv_file_path)
    with open(csv_file_path, mode='a' if file_exists else 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['id', 'img_path', 'caption'])

        # 헤더 작성 (파일이 없을 경우에만 작성)
        if not file_exists:
            writer.writeheader()

        # 데이터 작성
        writer.writerow({'id': entry['file_name'], 'img_path': entry['img_path'], 'caption': entry['caption']})


In [ ]:
### CSV 파일 경로 설정 및 캡션 생성 실행

# 압축 해제한 zip 파일명 정리
zip_file_names = []
for file_name in os.listdir(zip_folder_path):
    if file_name.endswith('.zip'):
        zip_file_names.append(os.path.splitext(file_name)[0])

for zip_file_name in zip_file_names:
    dir_path = os.path.join(extraction_base_path, zip_file_name)

    # 디렉토리 내 파일 검색
    if os.path.exists(dir_path):
        file_names = os.listdir(dir_path)
        total_images = len(file_names)  # 총 이미지 개수 계산

        for idx, file_name in enumerate(file_names, start=1):
            print(f"[{idx}/{total_images}] {file_name} 처리 중...")
            process_images_and_update_csv(dir_path, file_name, csv_file_path)

    else:
        print(f"디렉토리가 존재하지 않습니다: {dir_path}")

    print(f">> {zip_file_name} 내의 이미지 처리 완료!\n")


## 실행 중단 시

### 디렉토리별 진행률 파악

In [ ]:
### 디렉토리별 진행 정도 파악

# Training
csv_file_path = '/content/drive/My Drive/7th-project/data/captions_train.csv'
# Validation
# csv_file_path = '/content/drive/My Drive/7th-project/data/captions_val.csv'

# 특정 패턴 리스트 정의
patterns = [
    f"{i:02d}T_{j:02d}S"
    for i in range(1, 6)  # 01T ~ 05T
    for j in range(1, 4)  # 01S ~ 03S
]

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 결과 계산 및 출력
for pattern in patterns:
    t_part, s_part = pattern.split('_')  # "01T", "01S"로 분리

    # pattern의 각 부분(T, S)을 포함하는 키 필터링
    matching_keys = [
        key for key in directory_file_count.keys() if t_part in key and s_part in key
    ]
    total_value = sum(directory_file_count[key] for key in matching_keys)

    # CSV 데이터에서 pattern의 각 부분(T, S)을 모두 포함하는 데이터 개수 계산
    csv_count = df['id'].str.contains(t_part) & df['id'].str.contains(s_part)
    csv_count = csv_count.sum()

    # 결과 출력
    if total_value > 0:
        ratio = csv_count / total_value
        print(f"{pattern}: {csv_count}개 / {total_value} -> 비율: {ratio:.2f}")
    else:
        print(f"{pattern}: 관련 데이터 없음")

01T_01S: 2483개 / 2483 -> 비율: 1.00
01T_02S: 6713개 / 6713 -> 비율: 1.00
01T_03S: 3691개 / 3691 -> 비율: 1.00
02T_01S: 4827개 / 4827 -> 비율: 1.00
02T_02S: 0개 / 1678 -> 비율: 0.00
02T_03S: 0개 / 1282 -> 비율: 0.00
03T_01S: 0개 / 1125 -> 비율: 0.00
03T_02S: 0개 / 5371 -> 비율: 0.00
03T_03S: 0개 / 3509 -> 비율: 0.00
04T_01S: 0개 / 2708 -> 비율: 0.00
04T_02S: 0개 / 1308 -> 비율: 0.00
04T_03S: 402개 / 402 -> 비율: 1.00
05T_01S: 0개 / 1813 -> 비율: 0.00
05T_02S: 0개 / 1496 -> 비율: 0.00
05T_03S: 1595개 / 1595 -> 비율: 1.00


### 실행 중단 부분부터 시작하기

In [ ]:
### 실행 중단된 디렉토리 경로 설정
image_dir = "/content/data/TL_02T_자연탐구_01S_유아"

In [ ]:
### 실행 중단된 부분부터 재실행

# CSV 파일에서 기존에 저장된 이미지 ID 목록 가져오기
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
    existing_ids = set(df["id"].astype(str))  # 기존에 저장된 ID 목록
else:
    existing_ids = set()  # CSV 파일이 없으면 빈 집합

# 이미지 디렉토리에서 파일 목록 가져오기
all_images = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
image_ids = set(os.path.splitext(f)[0] for f in all_images)  # 확장자 제거 후 ID 리스트 생성

# 현 디렉토리 내에서 이미 처리된 파일 개수 계산
processed_images = image_ids & existing_ids  # 현재 디렉토리 내에 존재하면서, CSV에도 기록된 ID
num_processed_images = len(processed_images)

# 현 디렉토리 내에서 아직 처리되지 않은 파일 개수 계산
missing_images = list(image_ids - existing_ids)
total_missing_images = len(missing_images)

# 확인용 출력
print(f"총 이미지 개수 (현재 디렉토리 내): {len(image_ids)}")
print(f"이미 처리된 개수 (현재 디렉토리 내): {num_processed_images}")
print(f"다시 처리할 이미지 개수: {total_missing_images}")
print("-" * 50)
print()

# missing_images를 사용하여 다시 캡션 생성
for idx, img in enumerate(missing_images, start=1):
    img_path = os.path.join(image_dir, img + ".jpg")
    print(f"[{idx}/{total_missing_images}] {img} 처리 중...")
    process_images_and_update_csv(image_dir, img + ".jpg", csv_file_path)

print(f">> {image_dir} 내의 이미지 처리 완료!\n")

### (필요 시) 데이터 삭제

In [ ]:
### 실행 중단된 디렉토리의 전체 데이터 삭제

# 삭제할 디렉토리 패턴 설정
deleted_pattern = '01T_02S'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 삭제 대상 데이터 필터링
to_delete = df['id'].str.contains(deleted_pattern, na=False)  # deleted_pattern를 포함한 데이터
deleted_count = to_delete.sum()  # 삭제된 데이터 개수

# 삭제 수행
df_filtered = df[~to_delete]  # 삭제 대상이 아닌 데이터만 필터링

# CSV 파일 저장
df_filtered.to_csv(csv_file_path, index=False)

# 결과 출력
print(f"{deleted_pattern}를 포함한 데이터 {deleted_count}개 삭제 완료")
print(f"새로운 CSV 파일이 저장되었습니다: {csv_file_path}")

01T_02S를 포함한 데이터 560개 삭제 완료
새로운 CSV 파일이 저장되었습니다: /content/drive/My Drive/7th-project/data/captions_train.csv


In [ ]:
### csv 파일에서 마지막 n개의 열 삭제

# 삭제할 데이터 개수
deleted_num = 169

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 마지막 deleted_num개의 데이터 삭제
df = df[:-(deleted_num)]

# 수정된 데이터 저장
df.to_csv(csv_file_path, index=False)

print(f"CSV 파일에서 마지막 {deleted_num}개의 데이터를 삭제하고 저장했습니다.")

### 특정 디렉토리 지정하여 캡션 생성

In [ ]:
# 캡션 생성할 특정 디렉토리 입력
pending_zip_file_names = ['TL_02T_자연탐구_02S_초등_저학년']

for zip_file_name in pending_zip_file_names:
    dir_path = os.path.join(extraction_base_path, zip_file_name)

    # 디렉토리 내 파일 검색
    if os.path.exists(dir_path):
        file_names = os.listdir(dir_path)
        total_images = len(file_names)  # 총 이미지 개수 계산

        # 파일 순회
        for idx, file_name in enumerate(file_names, start=1):
            print(f"[{idx}/{total_images}] {file_name} 처리 중...")
            process_images_and_update_csv(dir_path, file_name, csv_file_path)

    else:
        print(f"디렉토리가 존재하지 않습니다: {dir_path}")

    print(f">> {zip_file_name} 내의 이미지 처리 완료!\n")

## 최종 데이터 개수 확인

In [ ]:
### CSV 파일 경로 설정: Training vs Validation

# Training
csv_file_path = '/content/drive/My Drive/7th-project/data/captions_train.csv'
# Validation
# csv_file_path = '/content/drive/My Drive/7th-project/data/captions_val.csv'

In [ ]:
### 데이터 수 확인 코드

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 데이터 개수 확인
row_count = len(df)
print(f"CSV 파일의 데이터 개수: {row_count}개")

In [ ]:
### 중복 데이터 확인 및 삭제

def find_and_remove_duplicate_ids(csv_file_path):

    if not csv_file_path or not os.path.exists(csv_file_path):
        print(f"[ERROR] 파일을 찾을 수 없습니다: {csv_file_path}")
        return

    # CSV 파일 읽기
    df = pd.read_csv(csv_file_path)

    # 'id' 열이 존재하는지 확인
    if 'id' not in df.columns:
        print("[ERROR] CSV 파일에 'id' 열이 존재하지 않습니다.")
        return

    # 중복된 id 찾기 (전체 개수 확인용)
    duplicate_ids = df[df.duplicated(subset=['id'], keep=False)]['id'].tolist()

    if duplicate_ids:
        print(f"[INFO] 중복된 id 개수: {len(set(duplicate_ids))}")
        print(f"[INFO] 중복된 id 리스트 (일부): {list(set(duplicate_ids))[:5]} ...")

        # 중복된 id 제거 (첫 번째 항목만 남기고 뒤의 데이터 삭제)
        df = df.drop_duplicates(subset=['id'], keep='first')

        # 변경된 데이터 저장 (덮어쓰기)
        df.to_csv(csv_file_path, index=False, encoding='utf-8')

        print(f"[SUCCESS] 중복 제거 완료! {csv_file_path} 업데이트됨.")
    else:
        print("[INFO] 중복된 id가 없습니다.")


find_and_remove_duplicate_ids(csv_file_path)

[INFO] 중복된 id가 없습니다.


In [ ]:
### 잘못 저장된 id 수정

def fix_ids_in_csv(input_csv, output_csv):
    """
    CSV 파일을 읽어서 id 컬럼의 값을 '/' 이후의 마지막 부분으로 수정한 후 새로운 CSV 파일로 저장
    """
    df = pd.read_csv(input_csv)
    if 'id' in df.columns:
        df['id'] = df['id'].astype(str).apply(lambda x: x.split('/')[-1])
        df.to_csv(output_csv, index=False)
    else:
        print("CSV 파일에 'id' 컬럼이 없습니다.")


input_csv = "/content/drive/My Drive/7th-project/data/captions_train.csv"  # 입력 CSV 파일 경로
output_csv = "/content/drive/My Drive/7th-project/data/captions_train.csv"  # 수정된 데이터를 저장할 CSV 파일 경로
fix_ids_in_csv(input_csv, output_csv)